В этой тетрадке:

- создаётся таблица sources
- создаётся таблица sources_types
- создаётся таблица languages

Данные взяты из таблицы
<code>[Источники2023](https://docs.google.com/spreadsheets/d/174Rv1TdwTOR4OW9zMtys04MVcLypzOv74aBCSBLJeEg/edit#gid=924994372)
</code>

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import csv

### Создание таблицы languages

In [ ]:
con = sqlite3.connect("Dictionary_Final.db")
cur = con.cursor()

In [ ]:
cur.execute("""
CREATE TABLE IF NOT EXISTS languages (
    lang_id INT, 
    language TEXT, 
    PRIMARY KEY (lang_id)
)
""")

con.commit()

In [ ]:
data_l = []
languages = ['греческий', 'латынь', 'немецкий', 'чешский']

for i in range(len(languages)):
    t = (i, languages[i])
    data_l.append(t)
    
data_l

In [ ]:
cur.executemany("INSERT INTO languages VALUES (?, ?)", data_l)
con.commit()

In [ ]:
df = pd.read_csv('sources.csv')
ids = [i for i in range(len(df))]

df.loc[(df['Lang'] == 'греческий'), 'Lang'] = 0
df.loc[(df['Lang'] == 'латынь'), 'Lang'] = 1
df.loc[(df['Lang'] == 'немецкий'), 'Lang'] = 2
df.loc[(df['Lang'] == 'чешский'), 'Lang'] = 3
df['Lang'].dropna()

### Создание таблицы sources_types

In [ ]:
cur.execute("""
CREATE TABLE IF NOT EXISTS sources_types (
    type_id INT, 
    type TEXT, 
    PRIMARY KEY (type_id)
)
""")

con.commit()

In [ ]:
data = []
types = ['Bib', 'Gomilet', 'Bog', 'Agiogr', 'Pov', 'Litur', 'Epi', 'Letop', 'Hron', 'Docum']

for i in range(len(types)):
    t = (i, types[i])
    data.append(t)
    
data

In [ ]:
cur.executemany("INSERT INTO sources_types VALUES (?, ?)", data)
con.commit()

### Создание таблицы sources

In [ ]:
cur.execute("""
CREATE TABLE IF NOT EXISTS sources_ (
    source_id INT PRIMARY KEY, 
    full name TEXT,
    abbr_name TEXT,
    lang_id INT,
    date_start_y INT,
    date_start_c INT,
    date_end_y INT,
    date_end_c INT,
    publication_date_y INT,
    publication_date_c INT,
    type_id INT,
    is_translated INT,
    FOREIGN KEY(lang_id) REFERENCES languages(lang_id),
    FOREIGN KEY(type_id) REFERENCES sources_types(type_id)
)
""")

con.commit()

In [ ]:
# список дат начала: год и век

date_start_y = []
date_start_c = []

for i in df['date_start']:
    if np.isnan(i):
        date_start_y.append('None')
        date_start_c.append('None')
    else:
        i_ = str(int(i))
        if len(i_) <= 2:
            date_start_c.append(int(i))
            date_start_y.append('None')
        elif len(i_) == 3:
            date_start_y.append(int(i))
            date_start_c.append(int(i_[0]) + 1)
        elif len(i_) == 4:
            date_start_y.append(int(i))
            date_start_c.append(int(i_[:2]) + 1)

In [ ]:
# список дат конца: год и век

date_end_y = []
date_end_c = []

for i in df['date_end']:
    if np.isnan(i):
        date_end_y.append('None')
        date_end_c.append('None')
    else:
        i_ = str(int(i))
        if len(i_) <= 2:
            date_end_c.append(int(i))
            date_end_y.append('None')
        elif len(i_) == 3:
            date_end_y.append(int(i))
            date_end_c.append(int(i_[0]) + 1)
        elif len(i_) == 4:
            date_end_y.append(int(i))
            date_end_c.append(int(i_[:2]) + 1)

In [ ]:
# переводной или непереводной источник

is_transl = []

for name in df['Lower']:
    name_ = str(name).split()
    if 'пер.' in name_:
        is_transl.append(1)
    else:
        is_transl.append(0)

In [ ]:
# сбор информации о жанрах источников

types_bd = []

with open('sources.csv', 'r', encoding='utf-8') as csvfile:
    filereader = csv.reader(csvfile, delimiter=',')
    for row in filereader:
        a = []
        row_ = row[7:17]
        for i in range(len(row_)):
            if row_[i] != '':
                a.append(i)
        types_bd.append(a)
        
types_new = types_bd[1:]

In [ ]:
df = df.astype({'publication_date_s': "Int64", 'publication_date_e': "Int64"})

In [ ]:
for_bd = {'source_id': ids,
          'full_name': df['Name'],
          'abbr_name': df['Sokr'],
          'lang_id': df['Lang'],
          'date_start_y': date_start_y,
          'date_start_c': date_start_c,
          'date_end_y': date_end_y,
          'date_end_c': date_end_c,
          'publication_date_s': df['publication_date_s'],
          'publication_date_e': df['publication_date_e'],
          'type_id': types_new,
          'is_translated': is_transl
         }

In [ ]:
df_new = pd.DataFrame(for_bd)
df_new.to_csv('df_final.csv', index=False, header=False)

In [ ]:
data_all = []

with open('df_final.csv', 'r', encoding='utf-8') as csvfile:
    filereader = csv.reader(csvfile, delimiter=',')
    for row in filereader:
        data_all.append(tuple(row))

data_all

In [ ]:
cur.executemany("INSERT INTO sources VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", data_all)
con.commit()

In [ ]:
# удаление дубликатов

cur.execute('SELECT * FROM sources')
rows = cur.fetchall()
unique_list = []

for row in rows:
    if row[1::] not in [i[1::] for i in unique_list]:
        unique_list.append(row)
    else:
        cur.execute(f"""
            DELETE FROM sources
            WHERE source_id={row[0]}
        """)
        
con.commit()

In [ ]:
con.close()